In [3]:
import pickle
import pandas as pd
from explainers.dce import DistributionalCounterfactualExplainer
import os

In [4]:
data_path = 'data/hotel_booking'

In [5]:
factual = pd.read_csv(os.path.join(data_path,'factual.csv'))
counterfactual = pd.read_csv(os.path.join(data_path,'counterfactual.csv'))

In [6]:
factual

,lead_time,booking_changes,is_canceled
0,5,0,0.180400
1,15,3,0.083647
2,102,0,0.469593
3,336,0,0.722741
4,304,0,0.691372
...,...,...,...
95,62,0,0.439348
96,219,0,0.558472
97,10,0,0.206225
98,67,0,0.455979


In [7]:
counterfactual

,lead_time,booking_changes,is_canceled
0,-185.689617,0.749337,0.011848
1,-211.033237,4.381826,0.003547
2,341.884936,-1.534492,0.986859
3,547.634998,-1.338327,0.990099
4,538.119511,-1.474443,0.992167
...,...,...,...
95,267.328818,-1.708036,0.988066
96,422.625213,-1.352951,0.985596
97,-170.423017,0.763208,0.014143
98,295.948675,-1.561551,0.985542
